<a href="https://colab.research.google.com/github/KirpaDmitriy/AIAlgsImplementation/blob/main/logical_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №3 "Логическая классификация"
## Студент: Кирпа Дмитрий
## Преподаватель: Мангараков Александр

## Подключение зависимостей

In [ ]:
from time import perf_counter
from typing import Any, Callable, Iterable

import numpy as np
import pandas as pd

from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [ ]:
np.random.seed(42)

## Данные

Был использован [набор](https://www.kaggle.com/datasets/valakhorasani/gym-members-exercise-dataset), содержащий показатели спортивных тренировок разных типов и показатели людей, их выполнявших:

In [ ]:
data = pd.read_csv("gym_members_exercise_tracking.csv")
data.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


И категориальные, и количественные признаки в наборе представлены:

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            973 non-null    int64  
 1   Gender                         973 non-null    object 
 2   Weight (kg)                    973 non-null    float64
 3   Height (m)                     973 non-null    float64
 4   Max_BPM                        973 non-null    int64  
 5   Avg_BPM                        973 non-null    int64  
 6   Resting_BPM                    973 non-null    int64  
 7   Session_Duration (hours)       973 non-null    float64
 8   Calories_Burned                973 non-null    float64
 9   Workout_Type                   973 non-null    object 
 10  Fat_Percentage                 973 non-null    float64
 11  Water_Intake (liters)          973 non-null    float64
 12  Workout_Frequency (days/week)  973 non-null    int

Добление в набор данных пропусков:

In [ ]:
data_with_nans = data.copy()

num_cells_to_nullify = 750

rows, cols = data.shape

random_indices = np.random.choice(rows * cols, num_cells_to_nullify, replace=False)

for index in random_indices:
    row = index // cols
    col = index % cols
    data_with_nans.iat[row, col] = np.nan

In [ ]:
data_with_nans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            926 non-null    float64
 1   Gender                         920 non-null    object 
 2   Weight (kg)                    932 non-null    float64
 3   Height (m)                     922 non-null    float64
 4   Max_BPM                        918 non-null    float64
 5   Avg_BPM                        918 non-null    float64
 6   Resting_BPM                    937 non-null    float64
 7   Session_Duration (hours)       924 non-null    float64
 8   Calories_Burned                916 non-null    float64
 9   Workout_Type                   929 non-null    object 
 10  Fat_Percentage                 920 non-null    float64
 11  Water_Intake (liters)          920 non-null    float64
 12  Workout_Frequency (days/week)  925 non-null    flo

In [ ]:
data.Experience_Level.value_counts()

,count
Experience_Level,
2,406
1,376
3,191


In [ ]:
X_train_classification_nans, X_test_classification_nans, y_train_classification_nans, y_test_classification_nans = train_test_split(data_with_nans.drop(columns=['Experience_Level']), data_with_nans.Experience_Level, test_size=0.25)
X_train_classification_cleaned_nans, X_test_classification_cleaned_nans, y_train_classification_cleaned_nans, y_test_classification_cleaned_nans = train_test_split(data_with_nans.dropna().drop(columns=['Experience_Level']), data_with_nans.dropna().Experience_Level, test_size=0.25)
X_train_classification, X_test_classification, y_train_classification, y_test_classification = train_test_split(data.drop(columns=['Experience_Level']), data.Experience_Level, test_size=0.25)

X_train_classification.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),BMI
354,25,Male,76.5,1.96,180,120,59,0.67,442.0,Strength,23.2,2.3,3,19.91
98,45,Female,59.4,1.51,169,142,65,1.15,735.0,Strength,33.1,1.9,4,26.05
951,49,Male,57.2,1.89,192,135,62,1.14,762.0,Cardio,21.1,3.5,2,16.01
825,58,Female,56.5,1.70,170,122,57,0.74,406.0,Yoga,27.0,2.7,2,19.55
569,53,Male,61.0,1.63,170,124,74,0.54,331.0,Cardio,20.0,3.4,3,22.96


In [ ]:
y_test_classification

,Experience_Level
138,1
279,1
361,1
929,1
629,2
...,...
516,2
544,2
773,1
749,2


In [ ]:
X_train_regression_nans, X_test_regression_nans, y_train_regression_nans, y_test_regression_nans = train_test_split(data_with_nans.drop(columns=['Calories_Burned']), data_with_nans.Calories_Burned, test_size=0.25)
X_train_regression, X_test_regression, y_train_regression, y_test_regression = train_test_split(data.drop(columns=['Calories_Burned']), data.Calories_Burned, test_size=0.25)

X_train_regression.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
809,54,Female,58.4,1.59,186,166,73,1.08,Cardio,32.5,1.9,2,1,23.10
492,52,Male,85.5,1.80,190,136,66,1.70,Strength,10.1,3.5,5,3,26.39
891,30,Male,90.0,1.66,165,152,71,1.32,Cardio,26.6,2.1,4,2,32.66
687,18,Male,125.9,1.67,172,153,60,1.46,Yoga,20.6,2.2,3,2,45.14
174,49,Male,79.2,1.72,194,128,69,1.46,Yoga,29.9,3.3,3,1,26.77


## ID3 дерево: классификация

### Реализация

#### Предикаты и их генерация

In [ ]:
class Predicate:
  def __init__(self, func: Callable, possible_values = {0, 1}):
    self.func = func
    self.possible_values = possible_values

  def __call__(self, x) -> Any:
    if pd.isnull(np.array(x)).any():
      raise ValueError("Nan значение передано в предикат")
    return self.func(x)

Предикаты генерируются, как проверка на равенство заданному значению для категориальных признаков и как проверка на принадлежность интервалу для числовых признаков. Интервалы генерируются, как полуотрезки равной длинны в заданном количестве на отрезке определения признака.

In [ ]:
def is_categorical(data, feature_index_or_name) -> bool:
    if isinstance(data, pd.DataFrame):
        return data.dtypes[feature_index_or_name].name == 'object' or data.dtypes[feature_index_or_name].name == 'category'
    else:
        # Для массивов предполагаем, что все строковые данные категориальные
        return isinstance(data[0][feature_index_or_name], str)

def make_predicate_function(feature, bin_range, eq_value=None) -> Callable:
    if eq_value is not None:
        return lambda x: x[feature] == eq_value
    else:
        return lambda x: bin_range[0] <= x[feature] < bin_range[1]

def generate_predicates(data, n_bins=3) -> Iterable:
    if isinstance(data, pd.DataFrame):
        columns = data.columns
    else:
        columns = range(len(data[0]))

    for feature_index_or_name in columns:
        feature_data = data[feature_index_or_name] if isinstance(data, pd.DataFrame) else list(zip(*data))[feature_index_or_name]
        if is_categorical(data, feature_index_or_name):
            unique_values = np.unique(feature_data, return_counts=False)
            for value in unique_values:
                yield make_predicate_function(feature_index_or_name, None, eq_value=value)
        else:
            min_val, max_val = min(feature_data), max(feature_data)
            bin_edges = np.linspace(min_val, max_val, num=n_bins+1)
            for i in range(len(bin_edges) - 1):
                yield make_predicate_function(feature_index_or_name, (bin_edges[i], bin_edges[i+1]))

data_frame = pd.DataFrame({
    'age': [25, 45, 35, 29],
    'gender': ['male', 'female', 'female', 'male'],
    'number': [50000, 54000, 62000, 59000]
})

data_list = [
    [25, 'male', 50000],
    [45, 'female', 54000],
    [35, 'female', 62000],
    [29, 'male', 59000]
]

predicate_iterator = generate_predicates(data_frame, n_bins=2)
for predicate in predicate_iterator:
    print([predicate(row) for index, row in data_frame.iterrows()])
print()
predicate_iterator = generate_predicates(data_list, n_bins=5)
for predicate in predicate_iterator:
    print([predicate(row) for row in data_list])

[True, False, False, True]
[False, False, True, False]
[False, True, True, False]
[True, False, False, True]
[True, True, False, False]
[False, False, False, True]

[True, False, False, False]
[False, False, False, True]
[False, False, True, False]
[False, False, False, False]
[False, False, False, False]
[False, True, True, False]
[True, False, False, True]
[True, False, False, False]
[False, True, False, False]
[False, False, False, False]
[False, False, False, True]
[False, False, False, False]


In [ ]:
max(1, -2, *[-1, -5])

1

#### Дерево

In [ ]:
class ID3ClassifierNode:
  def __init__(self):
    self.predicate = None
    self.children = {}
    self.children_probas = {}
    self.child_sub_samples = {}
    # лист от внутренней вершины отличается тем, что у листа class_label != None
    self.class_label = None
    self.major_class = None
    # после стрижки здесь может быть дочерняя вершина, которая заменяет данную
    self.prunned_by = None

  def predict_probas(self, x) -> dict[int, Any]:
    # если листовая вершина
    if self.class_label:
      # для листовой children probas - это вероятности классов
      return self.children_probas

    if self.predicate:
      try:
        predicate_value = self.predicate(x)
      except Exception:
        # βv (x) не определено =⇒ пропорциональное распределение
        probas = {}
        for child_predicate_value, child in self.children.items():
          child_probas = child.predict_probas(x)
          for class_label, class_label_proba in child_probas.items():
            probas.setdefault(class_label, 0)
            probas[class_label] += (
                class_label_proba *
                self.children_probas.get(child_predicate_value, 0)
                )
        return probas
      else:
        # значение из параллельной ветки не учитывается
        return self.children[predicate_value].predict_probas(x)
    else:
      if self.prunned_by:
        return self.prunned_by.predict_probas(x)
      raise ValueError(
          "Не задан ни класс для листовой, ни предикат для внутренней вершины"
          )

  def forward(self, x) -> Any:
    probas = self.predict_probas(x)
    return max(probas, key=probas.get)

  def accuracy(self, X, y) -> float:
    hits_n = 0
    for i in range(len(X)):
      hits_n += int(self.forward(X.to_numpy()[i]) == y.iloc[i])
    return hits_n / len(y)

  def prune(self, X, y) -> None:
    # X := подмножество объектов Xk, дошедших до текущей вершины

    # для всех v ∈ Vвнутр
    if self.class_label or not self.predicate:
      return

    # если Sv = ∅ то
    if not len(X):
      self.class_label = self.major_class
      self.children_probas = {self.class_label: 1}

    # Оцениваем ошибку текущего узла, если бы он был листом
    class_labels, counts = np.unique(y, return_counts=True)
    major_class = class_labels[np.argmax(counts)]
    error_leaf = np.sum(y != major_class)  # ошибка, если бы узел был листом

    # Рассчитываем ошибку для текущего дерева и каждого из детей
    v_error = 1 - self.accuracy(X, y)
    v_children = [1 - child.accuracy(X, y) for child in self.children.values()]

    # Сравниваем ошибки и принимаем решение о прунинге
    if (min_error := min(error_leaf, v_error, *v_children)) == error_leaf:
      # Превратить узел в лист
      self.class_label = major_class
      self.children_probas = {self.class_label: 1}
      return

    if min_error == v_error:
      for child in self.children.values():
        for child_predicate_value, child in self.children.items():
          mask = (
              np.apply_along_axis(self.predicate, 1, X) == child_predicate_value
          )
          sub_X = X[mask]
          sub_y = y[mask]
          child.prune(sub_X, sub_y)
      return

    for v_child, child in zip(v_children, self.children.values()):
      if min_error == v_child:
        self.predicate = None
        self.prunned_by = child
        return

  @staticmethod
  def major_method(x: np.array) -> Any:
    return stats.mode(x).mode

  def backward(
      self, X: np.array, y: np.array, betas: Iterable[Predicate], criterium
      ) -> None:
    betas = list(betas)
    # если все объекты из U лежат в одном классе c ∈ Y,
    # то вернуть новый лист v, cv := c
    if len(np.unique(y)) == 1:
      self.class_label = y[0]
      self.children_probas = {self.class_label: 1}
      self.major_class = y[0]
      return

    # если β(xi) не определено,
    # то при вычислении I(β,U) объект xi исключается из выборки U
    nan_mask = pd.isnull(y)
    X = X[~nan_mask]
    y = y[~nan_mask]
    self.major_class = self.major_method(y)

    calc_mask = pd.isnull(X).any(axis=1)
    X_calc = X[~calc_mask]
    y_calc = y[~calc_mask]

    # найти предикат с максимальной информативностью
    predicate = Predicate(
        max(betas, key=lambda beta: criterium(beta, X_calc, y_calc))
        )
    predicate_values = predicate.possible_values

    # разбить выборку на две части U = U0 ∪ U1 по предикату β
    for el, el_class in zip(X, y):
      # если предикат не вычислился,
      # то разбить на данном уровне по данному признаку невозможно
      try:
        predicate_el = predicate(el)
        self.child_sub_samples.setdefault(predicate_el, [])
        self.child_sub_samples[predicate_el].append((el, el_class))
      except Exception:
        ...

    total_length = sum(
        len(list(used_lines)) for used_lines in self.child_sub_samples.values()
        )

    # если U0 = ∅ или U1 = ∅,
    # то вернуть новый лист v, cv := Мажоритарный класс(U);
    for predicate_value in predicate_values:
      child_sub_sample = self.child_sub_samples.get(predicate_value)
      if not child_sub_sample:
        self.class_label = self.major_class
        self.children_probas = {self.class_label: 1}
        return

    # создать новую внутреннюю вершину v: βv := β
    self.predicate = predicate

    # построить левое поддерево: Lv := LearnID3(U0);
    # построить правое поддерево: Rv := LearnID3(U1)
    for predicate_value, child_sub_sample in self.child_sub_samples.items():
      child_node = self.__class__()
      child_X, child_y = zip(*child_sub_sample)
      child_node.backward(
          np.array(child_X),
          np.array(child_y),
          betas,
          criterium
      )
      self.children[predicate_value] = child_node
      self.children_probas[predicate_value] = len(child_X) / total_length


#### Критерий Донского

In [ ]:
def Donskoj_criterium(beta: Callable, X: np.array, y: np.array) -> float:
  # I(β, X) = #{(xi, xj) : β(xi) = β(xj) и yi = yj}
  count = 0
  n = len(X)
  for i in range(n):
    for j in range(i + 1, n):
      x_i, y_i = X[i], y[i]
      x_j, y_j = X[j], y[j]
      if beta(x_i) != beta(x_j) and y_i != y_j:
        count += 1
  return count

#### Многоклассовый энтропийный критерий

In [ ]:
def entropy(p: float) -> float:
  # -p log2(p)
  return -p * np.log2(p)

def multiclass_ent_criterium(beta: Callable, X: np.array, y: np.array) -> float:
  # I(β, Xl), Pc = #{xi: yi = c}, p = #{xi: β(xi) = 1}, h(z) = −z log2z
  unique_classes = np.unique(y)
  P_c = {c: np.sum(y == c) for c in unique_classes}
  p_c = {
      c: np.sum((y == c) & (np.array(beta(x) for x in X) == 1))
      for c in unique_classes
      }
  p = np.sum(np.array(beta(x) for x in X) == 1)
  l = len(X)

  res = 0
  for c in unique_classes:
    res += (
        entropy(P_c[c] / l)
        - (p / l) * entropy(P_c[c] / (p + 0.001))
        - ((l - p) / l) * entropy((P_c[c] - p_c[c]) / (l - p + 0.001))
        )
  return res

### Оценка качества

Данные без пропусков с разбиением числовых признаков на 20 полуотрезков:

In [ ]:
id3_classifier = ID3ClassifierNode()
id3_classifier.backward(
    X_train_classification.to_numpy(),
    y_train_classification.to_numpy(),
    generate_predicates(X_train_classification.to_numpy(), n_bins=20),
    Donskoj_criterium
)

In [ ]:
id3_classifier.children_probas

{True: 0.32510288065843623, False: 0.6748971193415638}

In [ ]:
id3_classifier.accuracy(X_test_classification, y_test_classification)

0.8073770491803278

Данные без пропусков с разбиением числовых признаков на 2 полуотрезка:

In [ ]:
id3_classifier_little_bins = ID3ClassifierNode()
id3_classifier_little_bins.backward(
    X_train_classification.to_numpy(),
    y_train_classification.to_numpy(),
    generate_predicates(X_train_classification.to_numpy(), n_bins=2),
    Donskoj_criterium
)

id3_classifier_little_bins.accuracy(X_test_classification, y_test_classification)

0.7540983606557377

Видно, что качество упало, но незначительно

Данные с пропусками с разбиением числовых признаков на 20 полуотрезков и критерием ветвления Донского:

In [ ]:
id3_classifier_nans = ID3ClassifierNode()
id3_classifier_nans.backward(
    X_train_classification_nans.to_numpy(),
    y_train_classification_nans.to_numpy(),
    generate_predicates(X_train_classification_nans.to_numpy(), n_bins=20),
    Donskoj_criterium
)

id3_classifier_nans.accuracy(X_test_classification_nans, y_test_classification_nans)

0.5368852459016393

Пропуски сильно влияют на качество предсказаний.

Данные с пропусками с разбиением числовых признаков на 20 полуотрезков и многоклассовым энтропийным критерием ветвления:

In [ ]:
id3_classifier_multi_ent = ID3ClassifierNode()
id3_classifier_multi_ent.backward(
    X_train_classification_nans.to_numpy(),
    y_train_classification_nans.to_numpy(),
    generate_predicates(X_train_classification_nans.to_numpy(), n_bins=2),
    multiclass_ent_criterium
)

id3_classifier_multi_ent.accuracy(X_test_classification_nans, y_test_classification_nans)

0.4098360655737705

Критерий Донского работает лучше на рассматриваемом наборе данных.

Данные с удалёнными строками с пропусками с разбиением числовых признаков на 20 полуотрезков и критерием ветвления Донского:

In [ ]:
id3_classifier_cleaned_nans = ID3ClassifierNode()
id3_classifier_cleaned_nans.backward(
    X_train_classification_cleaned_nans.to_numpy(),
    y_train_classification_cleaned_nans.to_numpy(),
    generate_predicates(X_train_classification_cleaned_nans.to_numpy(), n_bins=20),
    Donskoj_criterium
)

id3_classifier_cleaned_nans.accuracy(X_test_classification_cleaned_nans, y_test_classification_cleaned_nans)

0.7727272727272727

In [ ]:
X_train_classification_cleaned_nans.shape, X_train_classification_nans.shape

((328, 14), (729, 14))

Уменьшение набора данных почти в 2 раза не привело к значительному падению качества.

## ID3 дерево: регрессия

### Реализация

#### Критерий для регрессии

In [ ]:
def uncertainty(y) -> float:
  return np.mean((np.mean(y) - y) ** 2)

In [ ]:
def regression_criterium(beta: Callable, X: np.array, y: np.array) -> float:
  beta_X_values = np.apply_along_axis(beta, 1, X)
  f_u_b = 0
  for predicate_value in {0, 1}:
    if len(y_part := y[beta_X_values == predicate_value]):
      f_u_b += uncertainty(y) * (len(y_part) / len(y))
  return uncertainty(y) - f_u_b

#### Дерево

In [ ]:
class ID3RegressorNode(ID3ClassifierNode):
  def mse(self, X, y) -> float:
    return mean_squared_error(np.apply_along_axis(self.forward, 1, X), y)

  @staticmethod
  def major_method(x: np.array) -> float:
    return np.mean(x)

  def prune(self, X, y) -> None:
    # X := подмножество объектов Xk, дошедших до текущей вершины

    # для всех v ∈ Vвнутр
    if self.class_label or not self.predicate:
      return

    # если Sv = ∅ то
    if not len(X):
      self.class_label = self.major_class
      self.children_probas = {self.class_label: 1}

    # Оцениваем ошибку текущего узла, если бы он был листом
    error_leaf = mean_squared_error(np.array([self.major_class for _ in y]), y)

    # Рассчитываем ошибку для текущего дерева и каждого из детей
    v_error = self.mse(X, y)
    v_children = [child.mse(X, y) for child in self.children.values()]

    # Сравниваем ошибки и принимаем решение о прунинге
    if (min_error := min(error_leaf, v_error, *v_children)) == error_leaf:
      # Превратить узел в лист
      self.class_label = self.major_class
      self.children_probas = {self.class_label: 1}
      return

    if min_error == v_error:
      for child in self.children.values():
        for child_predicate_value, child in self.children.items():
          mask = (
              np.apply_along_axis(self.predicate, 1, X) == child_predicate_value
          )
          sub_X = X[mask]
          sub_y = y[mask]
          child.prune(sub_X, sub_y)
      return

    for v_child, child in zip(v_children, self.children.values()):
      if min_error == v_child:
        self.predicate = None
        self.prunned_by = child
        return


### Оценка качества

In [ ]:
id3_regressor = ID3RegressorNode()
id3_regressor.backward(
    X_train_regression.to_numpy(),
    y_train_regression.to_numpy(),
    generate_predicates(X_train_regression.to_numpy(), n_bins=20),
    regression_criterium
)

id3_regressor.mse(X_test_regression, y_test_regression)

14819.49534686533

In [ ]:
id3_regressor_little_bins = ID3RegressorNode()
id3_regressor_little_bins.backward(
    X_train_regression.to_numpy(),
    y_train_regression.to_numpy(),
    generate_predicates(X_train_regression.to_numpy(), n_bins=2),
    regression_criterium
)

id3_regressor_little_bins.mse(X_test_regression, y_test_regression)

62277.92797589185

## Редукция дерева

In [ ]:
X_train_classification_prun, X_test_classification_prun, y_train_classification_prun, y_test_classification_prun = train_test_split(data.drop(columns=['Experience_Level']), data.Experience_Level, test_size=0.35)
X_val_classification_prun, X_test_classification_prun, y_val_classification_prun, y_test_classification_prun = train_test_split(X_test_classification_prun, y_test_classification_prun, test_size=0.75)

In [ ]:
id3_classifier_cleaned_prun = ID3ClassifierNode()
id3_classifier_cleaned_prun.backward(
    X_train_classification_prun.to_numpy(),
    y_train_classification_prun.to_numpy(),
    generate_predicates(X_train_classification_prun.to_numpy(), n_bins=2),
    Donskoj_criterium
)

id3_classifier_cleaned_prun.accuracy(X_test_classification_prun, y_test_classification_prun)

0.7578125

In [ ]:
id3_classifier_cleaned_prun.prune(X_val_classification_prun, y_val_classification_prun)

In [ ]:
id3_classifier_cleaned_prun.accuracy(X_test_classification_prun, y_test_classification_prun)

0.7734375

Видно, что редукция повысила качество предсказаний. Дерево немного переобучилось.

## Библиотечная версия

In [ ]:
def lib_accuracy(lib_data, target_column, classification: bool = True):
  X_lib_train_classification, X_lib_test_classification, y_lib_train_classification, y_lib_test_classification = train_test_split(lib_data.drop(columns=[target_column]), lib_data[target_column], test_size=0.25)

  for cat_col in ['Workout_Type', 'Gender']:
    labelencoder = LabelEncoder()
    if target_column != cat_col:
      X_lib_train_classification[cat_col] = labelencoder.fit_transform(X_lib_train_classification[cat_col])
      X_lib_test_classification[cat_col] = labelencoder.transform(X_lib_test_classification[cat_col])
    else:
      y_lib_train_classification = labelencoder.fit_transform(y_lib_train_classification)
      y_lib_test_classification = labelencoder.transform(y_lib_test_classification)

  if classification:
    lib_id3_clf = DecisionTreeClassifier(criterion='entropy')

    start_time = perf_counter()
    lib_id3_clf.fit(X_lib_train_classification.dropna(), y_lib_train_classification.dropna())

    lib_clf_predict = lib_id3_clf.predict(X_lib_test_classification)
    return sum(lib_clf_predict == y_lib_test_classification) / len(y_test_classification), perf_counter() - start_time

  lib_id3_reg = DecisionTreeClassifier(criterion='entropy')

  start_time = perf_counter()
  lib_id3_reg.fit(X_lib_train_classification.dropna(), y_lib_train_classification.dropna())

  lib_clf_predict = lib_id3_reg.predict(X_lib_test_classification)
  return mean_squared_error(lib_clf_predict, y_lib_test_classification), perf_counter() - start_time

In [ ]:
lib_accuracy(data, 'Experience_Level')

(0.860655737704918, 0.015299848000722704)

Качество сопоставимо с реализованным вручную алгоритмом. При этом библиотечная версия требует категориальные признаки кодировать в числовые. Приведённая выше ручная реализация же допускает категориальные признаки любого типа данных.
Однако библиотечная версия работает гораздо быстрее ручной реализации: 10 мс против 5 мин.

In [ ]:
lib_accuracy(data_with_nans.dropna(), 'Experience_Level')

0.3770491803278688

Библиотечный алгоритм чувствителен к потере данных. Аналогичное уменьшение кол-ва тренировочной выборки для приведённой выше реализации не привели к значительному падению качества. Библиотечный же аналог стал хуже в 2 раза.

In [ ]:
lib_accuracy(data, 'Calories_Burned', classification=False)

14817.540983606557